In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

In [2]:
dti_folder = "/biodata/nyanovsky/datasets/dti/"

In [3]:
ChG_df = pd.read_csv(dti_folder+"ChG/ChG_df.csv")
ChCh_df = pd.read_csv(dti_folder+"ChCh/tani_net_05.csv")
GG_df = pd.read_csv(dti_folder+"/GG/pfam_proy_net.csv")

In [4]:
ChG_df.head()

,DrugID (PubChem CID),GeneID (NCBI)
0,155831,5243
1,24762158,213
2,24762158,506
3,24762158,563
4,24762158,13884


In [5]:
ChCh_df.head()

,CID1,CID2,Similarity
0,155831,148124,0.844262
1,155831,36314,0.822581
2,155831,9854073,0.609929
3,155831,6918473,0.578616
4,54692492,54680692,0.550725


In [6]:
ChCh_df.drop(columns="Similarity", inplace=True)

In [7]:
GG_df.head()

,src,trgt
0,1394,1395
1,1394,266977
2,1394,2692
3,1394,2696
4,1394,2740


In [8]:
ChG_df = ["C","G"]+ChG_df.astype(str)
ChCh_df = ["C","C"]+ChCh_df.astype(str)
GG_df = ["G","G"]+GG_df.astype(str)

ChCh_df["sorted_edge"] = ChCh_df.apply(lambda row: sorted(row), axis=1)
GG_df["sorted_edge"] = GG_df.apply(lambda row: sorted(row), axis=1)


ChG_df.drop_duplicates(inplace=True)
ChCh_df = ChCh_df.drop_duplicates(subset="sorted_edge").drop(columns="sorted_edge")
GG_df = GG_df.drop_duplicates(subset="sorted_edge").drop(columns="sorted_edge")

In [9]:
ChG_df.to_csv(dti_folder+"ChG/ChG_final_df.csv")
ChCh_df.to_csv(dti_folder+"ChCh/ChCh_final_df.csv")
GG_df.to_csv(dti_folder+"GG/GG_final_df.csv")

In [10]:
ChG_df.rename(columns={"DrugID (PubChem CID)":"src_id", "GeneID (NCBI)": "trgt_id" }, inplace=True)

ChCh_df.rename(columns={"CID1": "src_id", "CID2":"trgt_id"}, inplace=True)

GG_df.rename(columns={"src":"src_id", "trgt":"trgt_id"}, inplace=True)

In [75]:
ChG_df["edge_type"] = "chg"
ChG_df["src_node_type"] = "chem"
ChG_df["trgt_node_type"] = "gene"

In [76]:
ChCh_df["edge_type"] = "chch"
ChCh_df["src_node_type"] = "chem"
ChCh_df["trgt_node_type"] = "chem"

In [77]:
GG_df["edge_type"] = "gg"
GG_df["src_node_type"] = "gene"
GG_df["trgt_node_type"] = "gene"

In [78]:
edge_df = pd.concat([ChG_df, ChCh_df, GG_df])

In [79]:
edge_df

,src_id,trgt_id,edge_type,src_node_type,trgt_node_type
0,C155831,G5243,chg,chem,gene
1,C24762158,G213,chg,chem,gene
2,C24762158,G506,chg,chem,gene
3,C24762158,G563,chg,chem,gene
4,C24762158,G13884,chg,chem,gene
...,...,...,...,...,...
12644,G5649,G7143,gg,gene,gene
12645,G10417,G84870,gg,gene,gene
12646,G270,G271,gg,gene,gene
12647,G270,G272,gg,gene,gene


In [17]:
import networkx as nx

In [80]:
G = nx.from_pandas_edgelist(edge_df, source="src_id", target="trgt_id",edge_attr="edge_type")

In [81]:
def get_edgetype_subgraph(G: nx.Graph, edge_type: str) -> nx.Graph:
    rel_edges = [(u,v) for (u,v,r_dict) in G.edges.data() if r_dict["edge_type"]==edge_type]
    rel_subgraph = G.edge_subgraph(rel_edges).copy()
    return rel_subgraph

In [82]:
ChG_subgraph = get_edgetype_subgraph(G, "chg")
ChCh_subgraph = get_edgetype_subgraph(G, "chch")
GG_subgraph = get_edgetype_subgraph(G, "gg")

In [84]:
def node_deg_info(node):
    ChG_deg, ChCh_deg, GG_deg = 0,0,0
    if node in ChG_subgraph.nodes():
        ChG_deg = ChG_subgraph.degree(node)
    if node.startswith("C") and node in ChCh_subgraph.nodes():
        ChCh_deg = ChCh_subgraph.degree(node)
    elif node in GG_subgraph.nodes():
        GG_deg = GG_subgraph.degree(node)
    
    return pd.Series([ChG_deg, ChCh_deg, GG_deg])

In [88]:
node_df = pd.DataFrame()
node_df["node_id"] = list(G.nodes())
node_df[["ChG_deg","ChCh_deg", "GG_deg"]] = node_df["node_id"].apply(node_deg_info)
node_df["total_deg"] = node_df[["ChG_deg","ChCh_deg", "GG_deg"]].sum(axis=1)
node_df["node_type"] = node_df["node_id"].apply(lambda x: "chem" if x.startswith("C") else "gene")

In [89]:
# add src node index and trgt node index to edge_df as columns
node_idxs = {node_id:index for index, node_id in enumerate(node_df["node_id"])}
edge_df["src_node_index"] = edge_df["src_id"].apply(lambda id: node_idxs[id])
edge_df["trgt_node_index"] = edge_df["trgt_id"].apply(lambda id: node_idxs[id])

In [90]:
edge_df

,src_id,trgt_id,edge_type,src_node_type,trgt_node_type,src_node_index,trgt_node_index
0,C155831,G5243,chg,chem,gene,0,1
1,C24762158,G213,chg,chem,gene,2,3
2,C24762158,G506,chg,chem,gene,2,4
3,C24762158,G563,chg,chem,gene,2,5
4,C24762158,G13884,chg,chem,gene,2,6
...,...,...,...,...,...,...,...
12644,G5649,G7143,gg,gene,gene,11162,10028
12645,G10417,G84870,gg,gene,gene,9386,9499
12646,G270,G271,gg,gene,gene,4921,9345
12647,G270,G272,gg,gene,gene,4921,9346


In [91]:
from itertools import permutations
def make_undirected(df,column_pairs):
    new_combinations = [item for sublist in [list(permutations(column_pair,2)) for column_pair in column_pairs] for item in sublist]
    reversed_columns = {old:new for (old,new) in new_combinations}
    reversed_dataframe = df.rename(columns=reversed_columns)
    df = pd.concat([df,reversed_dataframe])#.drop_duplicates()
    return df

In [92]:
edge_df = make_undirected(edge_df,[("src_id", "trgt_id"), ("src_node_type","trgt_node_type"),
                                       ("src_node_index", "trgt_node_index")])

In [93]:
edge_df.to_csv(dti_folder+"edge_df.csv", index=False)
node_df.to_csv(dti_folder+"node_df.csv", index=False)

In [94]:
import pickle

In [95]:
pickle.dump(G, open(dti_folder+'dti_graph.pickle', 'wb'))